In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, input_nodes, hidden_1_nodes, hidden_2_nodes, output_nodes, learning_rate):
        
        self.name = gate_name
                
        # 2층 hidden1 layer 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 hidden2 layer 
        self.W3 = np.random.rand(hidden_1_nodes, hidden_2_nodes)
        self.b3 = np.random.rand(hidden_2_nodes)
                        
        # 4층 output layer 
        self.W4 = np.random.rand(hidden_2_nodes, output_nodes)
        self.b4 = np.random.rand(output_nodes)
        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
            
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.input_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        # cross-entropy 
        return  -np.sum( self.target_data*np.log(y + delta) + (1-self.target_data)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):    
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3, self.W4) + self.b4
        y = a4 = sigmoid(z4)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    
    # accuracy method 
    def accuracy(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(input_data)
        
        return accuracy_val
    
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self, input_data, target_data):
        
        self.input_data = input_data
        self.target_data = target_data
        
        f = lambda x : self.feed_forward()
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
        self.W4 -= self.learning_rate * numerical_derivative(f, self.W4)
    
        self.b4 -= self.learning_rate * numerical_derivative(f, self.b4)

In [3]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 1  # hidden 1 nodes
h2_nodes = 1  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    and_obj.train(and_xdata.reshape(4,2), and_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", and_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

AND object is created
step =  0 loss value =  3.176302034971994
step =  500 loss value =  2.1715911526705804
step =  1000 loss value =  0.2772799885752984
step =  1500 loss value =  0.07809333242216412
step =  2000 loss value =  0.04426803349799199
step =  2500 loss value =  0.030762748048402157
step =  3000 loss value =  0.023541181650781524
step =  3500 loss value =  0.0190551712985958
step =  4000 loss value =  0.016000881886060858
step =  4500 loss value =  0.013788387298526703
step =  5000 loss value =  0.012112339524795592
step =  5500 loss value =  0.010798984717283643
step =  6000 loss value =  0.009742223472287193
step =  6500 loss value =  0.00887362733538639
step =  7000 loss value =  0.008147093910727491
step =  7500 loss value =  0.007530433721210705
step =  8000 loss value =  0.007000492986353351
step =  8500 loss value =  0.006540192725388916
step =  9000 loss value =  0.006136660060403847
step =  9500 loss value =  0.00578000907462386
step =  10000 loss value =  0.00546

In [4]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

accuracy_ret = and_obj.accuracy(test_data)

print(and_obj.name, ' Accuracy => ', accuracy_ret)

AND 

AND  Accuracy =>  1.0


In [5]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 1  # hidden 1 nodes
h2_nodes = 1  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    or_obj.train(or_xdata.reshape(4,2), or_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", or_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

OR object is created
step =  0 loss value =  2.2467638961403607
step =  500 loss value =  1.8124111311222697
step =  1000 loss value =  0.13889186065071404
step =  1500 loss value =  0.059772324488268705
step =  2000 loss value =  0.03771258289142225
step =  2500 loss value =  0.02748081620375176
step =  3000 loss value =  0.021596234861593278
step =  3500 loss value =  0.01777941145537576
step =  4000 loss value =  0.015105353223788254
step =  4500 loss value =  0.013128522512636184
step =  5000 loss value =  0.011608091895732822
step =  5500 loss value =  0.01040257972039502
step =  6000 loss value =  0.00942345007851173
step =  6500 loss value =  0.008612482032029868
step =  7000 loss value =  0.007929826249636606
step =  7500 loss value =  0.007347292853578845
step =  8000 loss value =  0.006844380256560817
step =  8500 loss value =  0.006405822830954241
step =  9000 loss value =  0.006020020550290079
step =  9500 loss value =  0.00567800159359396
step =  10000 loss value =  0.0053

In [6]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

accuracy_ret = or_obj.accuracy(test_data)

print(or_obj.name, ' Accuracy => ', accuracy_ret)

OR 

OR  Accuracy =>  1.0


In [7]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 1  # hidden 1 nodes
h2_nodes = 1  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# OR Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    nand_obj.train(nand_xdata.reshape(4,2), nand_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", nand_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

NAND object is created
step =  0 loss value =  2.3013259197745866
step =  500 loss value =  2.250734313896495
step =  1000 loss value =  2.24687961898581
step =  1500 loss value =  2.2361689786300314
step =  2000 loss value =  1.9436487517824461
step =  2500 loss value =  0.15284422081557375
step =  3000 loss value =  0.061200154856353065
step =  3500 loss value =  0.03793678936016613
step =  4000 loss value =  0.027447513943583187
step =  4500 loss value =  0.021493100768094443
step =  5000 loss value =  0.017658959183291717
step =  5500 loss value =  0.014984827056252464
step =  6000 loss value =  0.013013827577042971
step =  6500 loss value =  0.011501019961551784
step =  7000 loss value =  0.01030333853364131
step =  7500 loss value =  0.009331638894918852
step =  8000 loss value =  0.008527491068652767
step =  8500 loss value =  0.007851004514252847
step =  9000 loss value =  0.007274017471609467
step =  9500 loss value =  0.0067760824527815285
step =  10000 loss value =  0.006341

In [8]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

accuracy_ret = nand_obj.accuracy(test_data)

print(nand_obj.name, ' Accuracy => ', accuracy_ret)

NAND 

NAND  Accuracy =>  1.0


#### XOR 검증 

In [9]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

# hyper-parameter
i_nodes = 2   # input nodes
h1_nodes = 2  # hidden 1 nodes , (h1, h2) = (1, 1) , (1, 2), (2,1) 않됨
h2_nodes = 2  # hidden 2 nodes
o_nodes = 1   # output nodes
iteration_count = 10001  # iteration count
lr = 1e-1  # learning rate, 1e-2 로 하면 손실함수가 감소하지 않음

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", i_nodes, h1_nodes, h2_nodes, o_nodes, lr)

start_time = datetime.now()

for step in range(iteration_count):

    xor_obj.train(xor_xdata.reshape(4,2), xor_tdata.reshape(4,1))
    
    if (step % 500 == 0):
        print("step = ", step, "loss value = ", xor_obj.loss_val())
              
end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

XOR object is created
step =  0 loss value =  2.9054922251636013
step =  500 loss value =  2.772570699687564
step =  1000 loss value =  2.7724490660915175
step =  1500 loss value =  2.7722894822059727
step =  2000 loss value =  2.772031363392373
step =  2500 loss value =  2.7715410482375997
step =  3000 loss value =  2.7704420502488754
step =  3500 loss value =  2.7672974403455184
step =  4000 loss value =  2.752145233285673
step =  4500 loss value =  2.4566527910854408
step =  5000 loss value =  1.8891668770864136
step =  5500 loss value =  0.10741848873078932
step =  6000 loss value =  0.04144797432344691
step =  6500 loss value =  0.024866071303022534
step =  7000 loss value =  0.017528679979190897
step =  7500 loss value =  0.013439248667541901
step =  8000 loss value =  0.010849316303487932
step =  8500 loss value =  0.009069507129479527
step =  9000 loss value =  0.007774936179674224
step =  9500 loss value =  0.006793079149246513
step =  10000 loss value =  0.00602409261389773



In [10]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

accuracy_ret = xor_obj.accuracy(test_data)

print(xor_obj.name, ' Accuracy => ', accuracy_ret)

XOR 

XOR  Accuracy =>  1.0
